5. 이전 대화를 기억하는 Chain 생성방법


RunnableWithMessageHistory를 활용하여 특정 유형의 작업(체인)에 메시지 기록을 추가하는 것

활용 예시
 - 대화형 챗봇 개발 : 사용자와의 대화 내역을 기반으로 챗봇의 응답을 조정
 - 복잡한 데이터 처리 : 데이터 처리 과정에서 이전 단계의 결과를 참조하여 다음 단계의 로직을 결정
 - 상태 관리가 필요한 애플리케이션 : 사용자의 이전 선택을 기억, 그에 따른 상호작용을 통해 정교한 애플리케이션 구현

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()


True

1. 일반 Chain에 대화기록만 추가

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.",
        ),
        
        MessagesPlaceholder(variable_name="chat_history"),   #대화 기록을 삽입할 자리를 미리 확보
        ("human", "#Question:\n{question}"),  
    ]
)

# llm 생성
llm = ChatOpenAI()

# 일반 Chain 생성
chain = prompt | llm | StrOutputParser()


MessagesPlaceholder(variable_name="chat_history")

 - 변수이기에 chat_history 말고도 conversatin_memory 등으로 변경 후 추후 코드에서 맞추어도 되긴 하지만,
 - lanchain의 메모리 기능 ConversationBufferMemory 와 자동연동은 안되기에 권장하지는 않음음

In [ ]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):               #session ids는 세션을 식별하기 위한 고유 키
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  
        store[session_ids] = ChatMessageHistory()         #store에 session ids가 없는경우 chatmessagehistory 객체를 통해 새로 만듦
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


chain_with_history = RunnableWithMessageHistory(             
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)


In [4]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "서울에서 인구수가 가장 많은 지역이 어디야?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'서울에서 인구가 가장 많은 지역은 송파구입니다. 송파구는 주거지역과 상업지역이 적절히 혼재되어 있고 편리한 교통 인프라로 인구가 밀집되어 있는 지역으로 알려져 있습니다.'

In [11]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "안녕! 6.25전쟁에 대해 설명해줘"},
    # 세션 ID 기준으로 대화를 기록합니다.
)

ValueError: Missing keys ['session_id'] in config['configurable'] Expected keys are ['session_id'].When using via .invoke() or .stream(), pass in a config; e.g., chain.invoke({'question': 'foo'}, {'configurable': {'session_id': '[your-value-here]'}})

In [5]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "그 다음은??"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'서울에서 인구가 많은 다음 지역으로는 강서구와 관악구가 나옵니다. 강서구와 관악구도 인구 밀집 지역으로 유명하며 서울 시내와 교통으로 잘 연결되어 있어 많은 사람들이 거주하고 있습니다.'

In [6]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "경기도에서는?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "kyh0719"}},
)

[대화 세션ID]: kyh0719


'경기도는 대한민국의 수도 서울을 중심으로 북쪽에 위치한 광역시 및 도이며, 인구 밀도가 높고 경제적으로 발달한 지역입니다. 경기도는 국내 GDP의 1/4 이상을 차지하는 대한민국의 경제 중심지이기도 합니다. 또한, 수도권 인구의 대다수가 거주하는 주요 지역 중 하나로, 서울과의 교통 및 경제적 관계가 밀접합니다.'

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# 단계 1: 문서 로드(Load Documents)
loader = PDFPlumberLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Previous Chat History:
{chat_history}

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [8]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 대화를 기록하는 RAG 체인 생성
rag_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)


In [9]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "삼성전자가 만든 생성형 AI 이름은?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)


[대화 세션ID]: rag123


"삼성전자가 만든 생성형 AI의 이름은 '삼성 가우스'입니다."

In [10]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "이전 답변을 영어로 번역해주세요."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)


[대화 세션ID]: rag123


'이전 답변을 영어로 번역하면 "The name of the generative AI created by Samsung Electronics is \'Samsung Gauss\'."입니다.'